# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv path ad name
csv = ("output_data_file/cities.csv")

# Read in csv file
cities_df = pd.read_csv(csv, index_col=0)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Set vairable and convert to float
humidity = cities_df["Humidity"].astype(float)

# Set max intenstity to the maximum humidity in dataframe
max_intense = cities_df["Humidity"].max().astype(float)

# Plot Heatmap
fig = gmaps.figure(layout={
        'width': '900px',
        'height': '500px',
        'padding': '3px',
        'border': '1px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_intense,
                                 point_radius=2, opacity=0.5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# set desired variables
upper_max_temp = 30
lower_max_temp = 25
max_wind_speed = 4
desired_cloudiness = 0

# Crate new dataframe based on the diresed variables outlined above
desired_cities_df = cities_df.loc[(cities_df["Max Temp"] > lower_max_temp) &
              (cities_df["Max Temp"] < upper_max_temp) &
              (cities_df["Wind Speed"] < max_wind_speed) &
              (cities_df["Cloudiness"] == desired_cloudiness)
             ]

# Create new dataframe that holds maximum 10 rows
hotel_df = desired_cities_df.iloc[0:10,:].copy()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# add search criteria
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
# Iterate through the lat and lng
for index, row in hotel_df.iterrows():
    
    # get lat,lng from dataframe
    lat_lng = str(f'{row["Lat"]},{row["Lng"]}')

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat_lng}"

    # Use the units term to return metric data for each lat, lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert to json
    response = requests.get(base_url, params=params).json()
    
 # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        
        # Adding collected data to hotel dataframe
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
                  
        # Add except 
    except (KeyError, IndexError):
        print("Hotel not found. Skipping....")
                  
        # Remove row if no data found
        hotel_df.drop(index, inplace=True)


Hotel not found. Skipping....


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,display_info_box = True, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='900px'))